# Bandit algorithms

*By*

*Henri Lemoine; 261056402; henri.lemoine@mail.mcgill.ca*

*Frederic Baroz; 261118133; frederic.baroz@mail.mcgill.ca*

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Constants

In [ ]:
SEED: int = 42
SAVE_FILES: bool = True
SHOW_GRAPHS: bool = True

## Helpers

In [3]:
def plot(
    data: np.array,
    title: str,
    main_labels: list[str],
    ax_titles: list[str],
    algs_info: list[tuple[str, str, int]],
    range_y: list[tuple[float, float]] = None,
    size: list[int] = (10, 10),
    fill_std: list[int] = None,
    filename: str = None,
    show: bool = False,
):
    """
    Args:
        data (np.array): All that needs to be plotted. Shape: (n_figs, n_curves, n_steps, n_runs).
        title (str): The title of the plot.
        main_labels (list[str]): A list of strings, the first element being the label of the x axis, and the rest being the labels of the y axes.
        ax_titles (list[str]): A list of titles for each subplot.
        algs_info (list[tuple[str, str, int]]): A list of tuples containing the information of each curve. Each tuple contains the following: (name, color, marker type). The marker type is an integer denoting the type of marker to be used. 0: normal full line, 1: dashed line, 2: scatter plot.
        range_y (list[tuple[float, float]], optional): A list of tuples of length n_figs, each denoting the range of the y axis of the corresponding subplot. Defaults to None.
        size (list[int], optional): The size of the plot. Defaults to (10, 10).
        fill_std (list[int], optional): A list of integers of length n_figs, each denoting whether and how to fill the standard deviation of the corresponding subplot. If None, no standard deviation will be filled. If 0, the standard deviation will be filled with a transparent color. If 1, the standard deviation will be a dashed line above and bellow the mean. If 2, just plot the standard deviation with a dashed line. Defaults to None.
        filename (str, optional): The name of the file to save the plot to. If none, the plot will not be saved. Defaults to None.
        show (bool, optional): Whether to show the plot or not. Defaults to False.

    Returns:
        None
    """
    nb_figs, nb_curves, nb_steps, nb_runs = data.shape
    range_x = (0, nb_steps)

    fig, axes = plt.subplots(nrows=nb_figs, ncols=1, figsize=size)
    fig.suptitle(title, fontsize=18, fontweight="bold", y=0.94)

    for i in range(nb_figs):
        if nb_figs == 1:
            ax = axes
        else:
            ax = axes[i]

        ax.set_title(ax_titles[i], fontsize=12, fontweight="bold", loc="left")

        for j in range(nb_curves):
            if algs_info[j][2] == 0:
                mean = np.mean(data[i, j, :, :], axis=1)
                std = np.std(data[i, j, :, :], axis=1) / np.sqrt(nb_runs)

                ax.plot(mean, label=algs_info[j][0], color=algs_info[j][1])

                if fill_std is not None:
                    if fill_std[i] == None:
                        pass
                    elif fill_std[i] == 0:
                        ax.fill_between(
                            np.arange(nb_steps),
                            mean - std,
                            mean + std,
                            alpha=0.2,
                            color=algs_info[j][1],
                        )
                    elif fill_std[i] == 1:
                        ax.plot(mean - std, color=algs_info[j][1], linestyle="--")
                        ax.plot(mean + std, color=algs_info[j][1], linestyle="--")
                    elif fill_std[i] == 2:
                        ax.plot(std, color=algs_info[j][1], linestyle="--")
                    else:
                        raise ValueError("Invalid fill_std value.")

            elif algs_info[j][2] == 1:
                ax.axhline(
                    y=np.mean(data[i, j, -1, :]),
                    label=algs_info[j][0],
                    color=algs_info[j][1],
                    linestyle="--",
                )

            elif algs_info[j][2] == 2:
                ax.scatter(
                    np.arange(nb_steps),
                    np.mean(data[i, j, :, :], axis=1),
                    s=10,
                    label=algs_info[j][0],
                    color=algs_info[j][1],
                )

            else:
                raise ValueError("Invalid marker type.")

        ax.legend(prop={"size": 10}, loc="right", bbox_to_anchor=(1.5, 0.5))

        if i == nb_figs - 1:
            ax.set_xlabel(main_labels[0], fontsize=14, labelpad=10)
        ax.set_ylabel(main_labels[i + 1], fontsize=14, labelpad=10)

        ax.set_xlim(range_x)
        if range_y is not None and range_y[i] is not None:
            ax.set_ylim(range_y[i])

        ax.locator_params(nbins=10, axis="x")
        ax.locator_params(nbins=5, axis="y")
        ax.grid()

    if filename is not None:
        plt.savefig(filename)  # , bbox_inches=Bbox([[0, 0], [13, 20]]))
    if show:
        plt.show()  # bbox_inches=Bbox([[0, 0], [13, 20]]))


## Question 1: k-armed bandit

Write a small simulator for a Bernoulli bandit with $k$ arms. The probability of success $p_i$ for each arm $i \in \{1,...,k\}$ should be provided as an input. The bandit should have a function called ”sample” which takes as input the index of an action and provides a reward sample. Recall that a Bernoulli bandit outputs either 1 or 0, drawn from a binomial distribution of parameter $p_k$. Test your code with 3 arms of parameters $q\ast = [0.5, 0.5 - \delta, 0.5 - 2\delta]$, with $\delta = 0.1$.
Generate and save a set of 50 samples for each action. For the test, plot one graph for each action, containing the reward values obtained over the 100 draws, the empirical mean of the values, and the true $q∗$ for each arm. Each graph will have an x-axis that goes to 50, two horizontal lines (true value and estimated value) and a set of points of value 0 and 1.

### 1.1. Class Definition

In [4]:
class MAB_Q1:
    def __init__(self, q_star):
        self.q_star = q_star
        self.k = len(q_star)  # number of arms

        self.q = np.zeros(self.k)  # estimated values
        self.n = np.zeros(self.k)  # number of times each arm was pulled

        self.reward_history = [[] for _ in range(self.k)]
        self.q_history = [[] for _ in range(self.k)]

    def sample(self, a_i):
        return np.random.binomial(1, self.q_star[a_i])

### 1.2. Getting the data

In [5]:
np.random.seed(SEED)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]

# Instantiating MAB with true expected values
mab = MAB_Q1(q_star)

nb_steps = 50

data_Q1 = np.zeros((len(q_star), 3, nb_steps, 1))

# Performing 50 samples for each action and storing the result in the reward history, and the estimated q values in the q history
for i in range(nb_steps):
    rewards = np.zeros(mab.k)
    for action_ix in range(mab.k):
        reward = mab.sample(action_ix)
        mab.n[action_ix] += 1
        mab.q[action_ix] += (reward - mab.q[action_ix]) / mab.n[action_ix]

        mab.reward_history[action_ix].append(reward)
        mab.q_history[action_ix].append(mab.q[action_ix])
        rewards[action_ix] = mab.sample(action_ix)

data_Q1[:, 0, :, 0] = mab.reward_history
data_Q1[:, 1, :, 0] = mab.q_history
data_Q1[:, 2, -1, 0] = q_star

# Additionally, to ensure we use the same samples in future tests, we can store the reward history
samples = mab.reward_history

### 1.3. Plotting the results

#### 1.3.1. First interpretation

Although the question asked for the plot to have 2 horizonal lines and a series of 1-0 points, we found informative to plot the averaged reward over time (blue line). Section 1.3.2. shows another set of plots with two horizontal lines (final action value for each action).

In [ ]:
data_Q1.shape
# (3, 3, 50, 1)
# 3 arms: 0, 1, 2
# 3 curves: reward, q, q_star
# 50 steps
# 1 run

In [ ]:
plot(
    data=data_Q1,
    title="Question 1\nK-armed Bandit (1)",
    main_labels=["Time steps", "Reward", "Reward", "Reward"],
    ax_titles=["Action 0", "Action 1", "Action 2"],
    algs_info=[("Reward values", "r", 2), ("Empirical mean", "b", 0), ("True q∗", "r", 1)],
    range_y=[(0, 1), (0, 1), (0, 1)],
    filename="q1_p1.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

#### 1.3.2. Second interpretation

Set of plot showing two horizontal lines as asked in the question with the blue dashed line indicating the final value estimation after 50 time steps.

In [ ]:
plot(
    data=data_Q1,
    title="Question 1\nK-armed Bandit (2)",
    main_labels=["Time steps", "Reward", "Reward", "Reward"],
    ax_titles=["Action 0", "Action 1", "Action 2"],
    algs_info=[
        ("Reward values", "r", 2),
        ("Empirical expected action value", "b", 1),
        ("True q∗", "r", 1),
    ],
    range_y=[(0, 1), (0, 1), (0, 1)],
    filename="q1_p2.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

## Question 2: Update and UpdateAvr

Code the rule for estimating action values discussed in lecture 2, with a fixed learning rate $\alpha$, in a function called $\text{update}$, and using the incremental computation of the mean presented in lecture 2, in a function called $\text{updateAvg}$. Using the previous data, plot for each action a graph showing the estimated $q$ value as a function of the number of samples, using averaging as well as $\alpha = 0.01$ and $\alpha = 0.1$, and the true value. Each graph should have three curves and a horizontal line.


### 2.1. Class definition

In [9]:
class MAB_Q2:
    def __init__(self, q_star: list[float], alpha: int = None):
        self.q_star = q_star
        self.k = len(q_star)

        self.alpha = alpha

        self.q = np.zeros(self.k)
        self.n = np.zeros(self.k)

        self.reward_history = [[] for _ in range(self.k)]
        self.q_history = [[] for _ in range(self.k)]

    def sample(self, action_ix: int) -> int:
        return np.random.binomial(1, self.q_star[action_ix])

    # Estimating action values with fixed alpha
    def update(self, action_ix: int, reward=None):
        # getting the reward from the binomial distribution of that action; if reward is given, use it instead
        if reward is None:
            reward = self.sample(action_ix)

        if self.alpha is None:
            raise ValueError("alpha is not defined")

        # updating the action value
        self.n[action_ix] += 1
        self.q[action_ix] += self.alpha * (reward - self.q[action_ix])

        self.reward_history[action_ix].append(reward)
        self.q_history[action_ix].append(self.q[action_ix])

    # Estimating action values with incrementally computed means
    def updateAvg(self, action_ix: int, reward=None):
        # getting the reward from the binomial distribution of that action; if reward is given, use it instead
        if reward is None:
            reward = self.sample(action_ix)

        # updating the action value
        self.n[action_ix] += 1
        self.q[action_ix] += (reward - self.q[action_ix]) / self.n[action_ix]

        self.reward_history[action_ix].append(reward)
        self.q_history[action_ix].append(self.q[action_ix])

### 2.2. Getting the data

In [10]:
np.random.seed(SEED)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]

# Instantiating 3 MAB (one for each update method)
mab_avg = MAB_Q2(q_star)
mab_001 = MAB_Q2(q_star, 0.01)
mab_01 = MAB_Q2(q_star, 0.1)

nb_steps = 50

data_Q2 = np.zeros((len(q_star), 4, nb_steps, 1))

# Taking the same 50 samples for each action as in Q1, and storing the result in the reward history, and the estimated q values in the q history
rewards = samples
for step_ix in range(nb_steps):
    for action_ix in range(mab_avg.k):
        mab_avg.updateAvg(action_ix, rewards[action_ix][step_ix])
        mab_001.update(action_ix, rewards[action_ix][step_ix])
        mab_01.update(action_ix, rewards[action_ix][step_ix])

data_Q2[:, 0, :, 0] = mab_avg.q_history
data_Q2[:, 1, :, 0] = mab_001.q_history
data_Q2[:, 2, :, 0] = mab_01.q_history
data_Q2[:, 3, -1, 0] = q_star

### 2.3. Plotting the results

In [ ]:
plot(
    data=data_Q2,
    title="Question 2\nUpdate & UpdateAvg",
    main_labels=["Time steps", "Action value", "Action value", "Action value"],
    ax_titles=["Action 0", "Action 1", "Action 2"],
    algs_info=[
        ("Empirical mean", "tab:blue", 0),
        (r"$\alpha = 0.01$", "tab:green", 0),
        (r"$\alpha = 0.1$", "tab:orange", 0),
        ("True q∗", "r", 1),
    ],
    range_y=[(0, 1), (0, 1), (0, 1)],
    filename="q2.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

## Question 3: Best averages experiment

Repeat the above experiment 100 times, starting with action value estimates of 0. Each run will still contain 100 samples for each action. Plot the same graph as above, but where the curves have the average and standard error over the 100 runs. 
Explain in 1-2 sentences what you observe. Which of the α values is better? How do they compare to averaging? If you wanted to optimize further, in what range of α would you look for better values?

Using MAB_Q2 from question 2.

### 3.1. Getting the data

In [12]:
np.random.seed(SEED)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 100
nb_steps = 100

data_Q3 = np.zeros((len(q_star), 4, nb_steps, nb_runs))

for run_ix in range(nb_runs):
    mab_avg = MAB_Q2(q_star, None)
    mab_001 = MAB_Q2(q_star, 0.01)
    mab_01 = MAB_Q2(q_star, 0.1)

    for step_ix in range(nb_steps):
        for action_ix in range(mab_avg.k):
            reward = mab_avg.sample(action_ix)
            mab_avg.updateAvg(action_ix, reward)
            mab_001.update(action_ix, reward)
            mab_01.update(action_ix, reward)
    data_Q3[:, 0, :, run_ix] = mab_avg.q_history
    data_Q3[:, 1, :, run_ix] = mab_001.q_history
    data_Q3[:, 2, :, run_ix] = mab_01.q_history
    data_Q3[:, 3, -1, run_ix] = q_star

### 3.2. Plotting the results

In [ ]:
plot(
    data=data_Q3,
    title="Question 3\nBest averages experiment",
    main_labels=["Time steps", "", f"Empirical expected value (averaged over {nb_runs} runs)", ""],
    ax_titles=["Action 0", "Action 1", "Action 2"],
    algs_info=[
        ("Empirical mean", "tab:blue", 0),
        (r"$\alpha = 0.01$", "tab:green", 0),
        (r"$\alpha = 0.1$", "tab:orange", 0),
        ("True q∗", "r", 1),
    ],
    range_y=[(0, 1), (0, 1), (0, 1)],
    fill_std=[0, 0, 0, None],
    filename="q3.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

As we can see, incremental averaging converges quickly; it is stable and its standard error is relatively small and continuously decreasing.

Though Alpha = 0.1 has a good initial convergence rate, its standard error remains large after 100 steps. For Alpha = 0.01, the convergence rate is slow, so we should expect longer nb_steps to provide better results. It has a small standard error.

## Question 4: $\epsilon$-greedy Algorithm

Code the ε-greedy algorithm discussed in class, with averaging updates, with ε provided as an input. You will run 100 independent runs, each consisting of 1000 time steps. Plot the following graphs:

a.  The reward received over time, averaged at each time step over the 100 independent runs (with no smoothing over the time steps), and the standard error over the 100 runs

b.  The fraction of runs (out of 100) in which the first action (which truly is best) is also estimated best based on the action values

c.  The instantaneous regret l_t (as discussed in lecture 3) (averaged over the 100 runs)

d.  The total regret L_t up to time step t (as discussed in lecture 3) (averaged over the 100 runs)

Generate this set of graphs, for the following values of ε: 0, 1/8, 1/4, 1/2, 1.

Explain what you observe in the graphs and discuss the effect of ε you observe.

**Remarks**
- *** I definitely think the fill for standard error was nice for Q3 but here... what we could do is provide this as first version and provide alternate view with only the means and SE deviation as second (dotted/dashed) lines since they do not appear at the same value (similar to what I originaly did)... so a set of 4 graphs with only reward  with SE, and the 1 additional graph for reward with alternative way of plotting SE (like absolute value of SE).

### 4.1. Class definition

In [14]:
class EpsilonGreedy_MAB_Q4(MAB_Q2):
    def __init__(self, q_star: list[float], epsilon: int = 0, alpha: int = None):
        super().__init__(q_star, alpha)
        self.epsilon = epsilon

        self.currentStep = 0
        self.total_reward = 0
        self.total_regret = 0
        self.fraction_correct = 0

        self.mean_reward_history = []
        self.fraction_correct_history = []
        self.instant_regret_history = []
        self.total_regret_history = []

    def draw_sample(self):
        action_ix = self.epsilon_greedy_choice()
        self.update(action_ix)

    def epsilon_greedy_choice(self) -> int:
        # Choosing action with epsilon-greedy policy
        e_greedy_choice = np.random.choice(["random", "greedy"], p=[self.epsilon, 1 - self.epsilon])
        if e_greedy_choice == "random":
            return np.random.choice(self.k)
        elif e_greedy_choice == "greedy":
            return np.random.choice(np.flatnonzero(self.q == self.q.max()))

    def update(self, action_ix: int, reward=None) -> None:
        # getting the reward from the binomial distribution of that action; if reward is given, use it instead
        if reward is None:
            reward = self.sample(action_ix)

        self.n[action_ix] += 1
        if self.alpha is None:
            self.q[action_ix] += (reward - self.q[action_ix]) / self.n[action_ix]
        else:
            self.q[action_ix] += self.alpha * (reward - self.q[action_ix])

        self.q_history.append(self.q)

        self.total_reward += reward
        self.mean_reward_history.append(reward)  # this will be averaged over nb_runs in plot
        self.instant_regret_history.append(max(self.q_star) - self.q_star[action_ix])
        self.total_regret += self.instant_regret_history[-1]
        self.total_regret_history.append(self.total_regret)
        self.fraction_correct_history.append(
            1 if action_ix == np.argmax(self.q_star) else 0
        )  # this will be averaged over nb_runs in plot

        self.currentStep += 1

### 4.2. Getting the data

In [15]:
np.random.seed(SEED)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 100
nb_steps = 1000

epsilons = [0, 1 / 8, 1 / 4, 1 / 2, 1]

data_Q4 = np.zeros((4, len(epsilons), nb_steps, nb_runs))

for run_ix in range(nb_runs):
    for e_ix, epsilon in enumerate(epsilons):
        mab = EpsilonGreedy_MAB_Q4(q_star, epsilon)
        for step_ix in range(nb_steps):
            mab.draw_sample()
        data_Q4[0, e_ix, :, run_ix] = mab.mean_reward_history
        data_Q4[1, e_ix, :, run_ix] = mab.fraction_correct_history
        data_Q4[2, e_ix, :, run_ix] = mab.instant_regret_history
        data_Q4[3, e_ix, :, run_ix] = mab.total_regret_history

### 4.3. Plotting the results

#### 4.3.1. With standard errors

In [ ]:
plot(
    data=data_Q4,
    title="Question 5\nepsilon-greedy bandit",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (r"$\epsilon = 0$", "tab:blue", 0),
        (r"$\epsilon = \frac{1}{8}$", "tab:green", 0),
        (r"$\epsilon = \frac{1}{4}$", "tab:orange", 0),
        (r"$\epsilon = \frac{1}{2}$", "tab:red", 0),
        (r"$\epsilon = 1$", "tab:purple", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q4[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q4[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[0, None, None, None],
    filename="q4.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

#### 4.3.2 Without standard errors

TO CHANGE; I'M NOT SURE WHAT YOU WERE TRYING TO SAY

In [ ]:
plot(
    data=data_Q4[:1, 1:2, :, :],
    title="Question 4\nEpsilon-greedy bandit (no standard errors)",
    main_labels=["Time steps", "Mean reward"],
    ax_titles=[""],
    algs_info=[(r"$\epsilon = 0$", "tab:blue", 0)],
    #    (r'$\epsilon = \frac{1}{8}$', 'tab:green', 0),
    #    (r'$\epsilon = \frac{1}{4}$', 'tab:orange', 0),
    #    (r'$\epsilon = \frac{1}{2}$', 'tab:red', 0),
    #    (r'$\epsilon = 1$', 'tab:purple', 0)],
    size=(10, 10),
    range_y=[(0, 1)],
    fill_std=[None],
    filename="q4_2.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

## Question 5: Experiment

For $\epsilon = 1/4$ and $\epsilon = 1/8$, plot the same graphs for $\alpha = 0.1$, $\alpha = 0.01$, $\alpha = 0.001$ and averaging.

Explain in 2 sentences what you observe.


We use the class MAB_Q4 of question 4 which already contains the update() method for fixed stepsize parameter alpha.

Again we draw plots for mean reward + SE, % optimal actions, instantaneous and total regret. Each graph contains 8 curves (as understood from the assignement), one for each combination of epsilon (1/8 and 1/4) and alpha (0.1, 0.01, 0.001) + incremental averaging.

### 5.1. Getting the data

In [18]:
np.random.seed(0)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 100
nb_steps = 1000

epsilons = [1 / 8, 1 / 4]
alphas = [0.1, 0.01, 0.001, None]

data_Q5 = np.zeros((4, len(epsilons) * len(alphas), nb_steps, nb_runs))

for run_ix in range(nb_runs):
    for e_ix, epsilon in enumerate(epsilons):
        for a_ix, alpha in enumerate(alphas):
            mab = EpsilonGreedy_MAB_Q4(q_star, epsilon, alpha)
            for step_ix in range(nb_steps):
                mab.draw_sample()
            data_Q5[0, e_ix * len(alphas) + a_ix, :, run_ix] = mab.mean_reward_history
            data_Q5[1, e_ix * len(alphas) + a_ix, :, run_ix] = mab.fraction_correct_history
            data_Q5[2, e_ix * len(alphas) + a_ix, :, run_ix] = mab.instant_regret_history
            data_Q5[3, e_ix * len(alphas) + a_ix, :, run_ix] = mab.total_regret_history

### 5.2. Plotting the results (with std error)

In [ ]:
plot(
    data=data_Q5,
    title="Question 5\nepsilon-greedy bandit with alpha updates",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (r"$\epsilon = 1/8, \alpha = 0.1$", "tab:blue", 0),
        (r"$\epsilon = 1/8, \alpha = 0.01$", "tab:green", 0),
        (r"$\epsilon = 1/8, \alpha = 0.001$", "tab:orange", 0),
        (r"$\epsilon = 1/8$, Empirical mean", "tab:red", 0),
        (r"$\epsilon = 1/4, \alpha = 0.1$", "tab:purple", 0),
        (r"$\epsilon = 1/4, \alpha = 0.01$", "tab:brown", 0),
        (r"$\epsilon = 1/4, \alpha = 0.001$", "tab:pink", 0),
        (r"$\epsilon = 1/4$, Empirical mean", "tab:gray", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q5[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q5[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[0, None, None, None],
    filename="q5.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

### 5.2. Plotting the results (without std error)

In [20]:
# plot(
#     data=data_Q5,
#     title='Question 5\nEpsilon-greedy bandit',
#     main_labels=['Time steps', 'Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     ax_titles=['Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     algs_info=[('Epsilon = 1/8, Alpha = 0.1', 'tab:blue', 0), ('Epsilon = 1/8, Alpha = 0.01', 'tab:green', 0), ('Epsilon = 1/8, Alpha = 0.001', 'tab:orange', 0), ('Epsilon = 1/8, Alpha = None', 'tab:red', 0), ('Epsilon = 1/4, Alpha = 0.1', 'tab:purple', 0), ('Epsilon = 1/4, Alpha = 0.01', 'tab:brown', 0), ('Epsilon = 1/4, Alpha = 0.001', 'tab:pink', 0), ('Epsilon = 1/4, Alpha = None', 'tab:gray', 0)],
#     size=(10, 20),
#     fill_std=False,
#     filename='Files/q5_no_std_errors.png'
# )

## Question 6: UCB Algorithm

Write a function that implements the UCB algorithm. Set c = 2.

Plot the same graphs as above for α = 0.1, α = 0.01, α = 0.001 and averaging.

Explain briefly the behavior you observe.

### 6.1. Class definition

First we define in MAB_Q6 a multi arm bandit with the UCB policy for action selection (inside the method play()).

In [21]:
class UCB_MAB_Q6(EpsilonGreedy_MAB_Q4):
    def __init__(self, q_star: list[float], alpha: int = None, c: int = 2):
        super().__init__(q_star, alpha)
        self.c = c

    def draw_sample(self):
        action_ix = self.ucb_choice()
        self.update(action_ix)

    def ucb_choice(self):
        # Choose the greedy UCB action. Break ties randomly
        ucb_choice = np.argmax(self.q + (self.c * np.sqrt(np.log(self.currentStep) / self.n)))
        return np.random.choice(np.flatnonzero(self.q == self.q[ucb_choice]))

### 6.2. Getting the data

In [22]:
np.random.seed(0)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 100
nb_steps = 1000

alphas = [0.1, 0.01, 0.001, None]
c = 2

data_Q6 = np.zeros((4, len(alphas), nb_steps, nb_runs))

for run_ix in range(nb_runs):
    for a_ix, alpha in enumerate(alphas):
        mab = UCB_MAB_Q6(q_star, alpha, c)
        for action_ix in range(len(q_star)):
            mab.update(action_ix)
        for step_ix in range(nb_steps - len(q_star)):
            mab.draw_sample()
        data_Q6[0, a_ix, :, run_ix] = mab.mean_reward_history
        data_Q6[1, a_ix, :, run_ix] = mab.fraction_correct_history
        data_Q6[2, a_ix, :, run_ix] = mab.instant_regret_history
        data_Q6[3, a_ix, :, run_ix] = mab.total_regret_history

### 6.3. Plotting the results (with std error)

In [ ]:
plot(
    data=data_Q6,
    title="Question 6\nUCB bandit",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (rf"$\alpha = {alphas[0]}$, $C = {c}$", "tab:blue", 0),
        (rf"$\alpha = {alphas[1]}$, $C = {c}$", "tab:green", 0),
        (rf"$\alpha = {alphas[2]}$, $C = {c}$", "tab:orange", 0),
        (rf"Empirical mean, $C = {c}$", "tab:red", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q6[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q6[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[0, None, None, None],
    filename="q6.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

### 6.3. Plotting the results (without std error)

In [24]:
# plot(
#     data=data_Q6,
#     title='Question 6\nUCB bandit',
#     main_labels=['Time steps', 'Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     ax_titles=['Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     algs_info=[('Alpha = 0.1, C = 2', 'tab:blue', 0), ('Alpha = 0.01, C = 2', 'tab:green', 0), ('Alpha = 0.001, C = 2', 'tab:orange', 0), ('Alpha = None, C = 2', 'tab:red', 0)],
#     size=(10, 20),
#     fill_std=False,
#     filename='Files/q6_no_std_errors.png'
# )

## Question 7: Thomson Sampling Algorithm

Write a function that implements the Thompson sampling.

Plot the same graphs as above for $\alpha = 0.1$, $\alpha = 0.01$, $\alpha = 0.001$ and averaging.

Explain briefly the behavior you observe.

### 7.1. Class definition

In [25]:
class ThomsonSampling_MAB_Q7(EpsilonGreedy_MAB_Q4):
    def __init__(self, q_star: list[float], alpha: int = None):
        super().__init__(q_star, alpha)

        # storing the number of successes for each actions to parametrize beta distribution
        # in the case of bernouilli, for a given action, number of failure is number of times action has been selected minus number of success
        self.nb_successes = np.zeros(len(q_star))

    def draw_sample(self):
        action_ix = self.thomson_sampling_choice()
        self.update(action_ix)
        self.nb_successes[action_ix] += reward

    def thomson_sampling_choice(self):
        # Choose the greedy thomson sampling action. Break ties randomly
        thomson_sampling_choice = np.argmax(
            np.random.beta(self.nb_successes + 1, self.n - self.nb_successes + 1)
        )
        return np.random.choice(np.flatnonzero(self.q == self.q[thomson_sampling_choice]))
        # ucb_choice = np.argmax(self.q + (self.c * np.sqrt(np.log(self.currentStep)/self.n)))
        # return np.random.choice(np.flatnonzero(self.q == self.q[ucb_choice]))

        # thomson_choice = np.random.beta(self.nb_successes + 1, self.n - self.nb_successes + 1)
        # return np.random.choice(np.flatnonzero(thomson_choice == thomson_choice.max()))

    # def update(self, action_ix: int, reward=None) -> float:
    #   # getting the reward from the binomial distribution of that action; if reward is given, use it instead
    #   if reward is None:
    #     reward = self.sample(action_ix)
    #   self.nb_successes[action_ix] += reward

    #   self.n[action_ix] += 1
    #   if self.alpha is None:
    #     self.q[action_ix] += (reward - self.q[action_ix]) / self.n[action_ix]
    #   else:
    #     self.q[action_ix] += self.alpha * (reward - self.q[action_ix])

    #   self.q_history.append(self.q)

    #   self.total_reward += reward
    #   self.mean_reward_history.append(self.total_reward / (self.currentStep + 1))
    #   self.instant_regret_history.append(max(self.q_star) - self.q_star[action_ix])
    #   self.total_regret += self.instant_regret_history[-1]
    #   self.total_regret_history.append(self.total_regret)
    #   is_correct_action = 1 if action_ix == np.argmax(self.q_star) else 0
    #   self.fraction_correct = (self.fraction_correct * self.currentStep + is_correct_action) / (self.currentStep + 1)
    #   self.fraction_correct_history.append(self.fraction_correct)

    #   self.currentStep += 1


# class ThompsonSampling(Bandit):
#     def __init__(self, p, seed=0):
#         super().__init__(p, seed)
#         self.N = np.zeros((self.k, 2))

#     def run_thompson_sampling(self, n):
#         for i in range(n):
#             action = np.argmax(np.random.beta(self.N + 1, 1 - self.N))
#             reward, regret = self.sample(action)
#             self.N[action] += reward
#             self.total_regret += regret
#             self.regret_history.append(self.total_regret)

### 7.2. Getting the data

In [26]:
np.random.seed(0)

delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 30
nb_steps = 1000

alphas = [0.1, 0.01, 0.001, None]

data_Q7 = np.zeros((4, len(alphas), nb_steps, nb_runs))

for run_ix in range(nb_runs):
    for a_ix, alpha in enumerate(alphas):
        mab = ThomsonSampling_MAB_Q7(q_star, alpha)
        for step_ix in range(nb_steps):
            mab.draw_sample()
        data_Q7[0, a_ix, :, run_ix] = mab.mean_reward_history
        data_Q7[1, a_ix, :, run_ix] = mab.fraction_correct_history
        data_Q7[2, a_ix, :, run_ix] = mab.instant_regret_history
        data_Q7[3, a_ix, :, run_ix] = mab.total_regret_history

### 7.3. Plotting the results (with std error)

In [ ]:
plot(
    data=data_Q7,
    title="Question 6\nThomson Sampling bandit",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (rf"$\alpha = {alphas[0]}$, $C = {c}$", "tab:blue", 0),
        (rf"$\alpha = {alphas[1]}$, $C = {c}$", "tab:green", 0),
        (rf"$\alpha = {alphas[2]}$, $C = {c}$", "tab:orange", 0),
        (rf"Empirical mean, $C = {c}$", "tab:red", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q7[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q7[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[0, None, None, None],
    filename="q7.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

### 7.4. Plotting the results (without std error)

In [28]:
# plot(
#     data=thomson_data_Q7,
#     title='Question 6\nThomson Sampling bandit',
#     main_labels=['Time steps', 'Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     ax_titles=['Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     algs_info=[('Alpha = 0.1, C = 2', 'tab:blue', 0), ('Alpha = 0.01, C = 2', 'tab:green', 0), ('Alpha = 0.001, C = 2', 'tab:orange', 0), ('Alpha = None, C = 2', 'tab:red', 0)],
#     size=(10, 20),
#     fill_std=False,
#     filename='Files/q7_no_std_errors.png'
# )

## Question 8: Algorithmic comparison

For each of the algorithms, pick the best hyper-parameter combination you have observed (explain how you decided what ”best” means). Plot together the curves for this setting. Comment on the relative behavior of the different algorithms.

For the purposes of this question, the "best" hyperparameter(s) are the ones that end up with the best mean regret after 1000 steps.

### 8.1. Getting the data

In [29]:
delta = 0.1
q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
nb_runs = 100
nb_steps = 1000

data_Q8 = np.zeros((4, 3, nb_steps, nb_runs))

# Run the algorithms
for run_ix in range(nb_runs):
    best_eg_epsilon = 1 / 8
    best_eg_alpha = None
    best_ucb_alpha = 0.01
    best_ucb_c = 2
    best_thomson_alpha = 0.01

    mab_eg = EpsilonGreedy_MAB_Q4(q_star, epsilon=best_eg_epsilon, alpha=best_eg_alpha)
    mab_ucb = UCB_MAB_Q6(q_star, alpha=best_ucb_alpha, c=best_ucb_c)
    for action_ix in range(len(q_star)):
        mab_ucb.update(action_ix)
    mab_thomson = ThomsonSampling_MAB_Q7(q_star, alpha=best_thomson_alpha)

    for step_ix in range(nb_steps):
        mab_eg.draw_sample()
        if step_ix >= 3:
            mab_ucb.draw_sample()
        mab_thomson.draw_sample()

    data_Q8[0, 0, :, run_ix] = mab_eg.mean_reward_history
    data_Q8[1, 0, :, run_ix] = mab_eg.fraction_correct_history
    data_Q8[2, 0, :, run_ix] = mab_eg.instant_regret_history
    data_Q8[3, 0, :, run_ix] = mab_eg.total_regret_history

    data_Q8[0, 1, :, run_ix] = mab_ucb.mean_reward_history
    data_Q8[1, 1, :, run_ix] = mab_ucb.fraction_correct_history
    data_Q8[2, 1, :, run_ix] = mab_ucb.instant_regret_history
    data_Q8[3, 1, :, run_ix] = mab_ucb.total_regret_history

    data_Q8[0, 2, :, run_ix] = mab_thomson.mean_reward_history
    data_Q8[1, 2, :, run_ix] = mab_thomson.fraction_correct_history
    data_Q8[2, 2, :, run_ix] = mab_thomson.instant_regret_history
    data_Q8[3, 2, :, run_ix] = mab_thomson.total_regret_history

### 8.2. Plotting the results (with std error)

In [ ]:
plot(
    data=data_Q8,
    title="Question 8\nEpsilon Greedy, UCB and Thomson Sampling bandits",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (r"$\epsilon$-greedy: $\epsilon = 1/8$, Empirical mean", "tab:blue", 0),
        (r"UCB, $\alpha = 0.01$", "tab:green", 0),
        (r"Thomson Sampling, $\alpha = 0.01$", "tab:orange", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q8[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q8[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[0, None, None, None],
    filename="q8.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

### 8.2. Plotting the results (without std error)

In [31]:
# plot(
#     data=Q8_data,
#     title='Question 8\nEpsilon Greedy, UCB and Thomson Sampling bandits',
#     main_labels=['Time steps', 'Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     ax_titles=['Average reward', 'Fraction of correct actions', 'Instant regret', 'Total regret'],
#     algs_info=[('Epsilon Greedy, epsilon = 1/8', 'tab:blue', 0), ('UCB, alpha = None', 'tab:green', 0), ('Thomson Sampling, alpha = None', 'tab:orange', 0)],
#     size=(10, 20),
#     fill_std=False,
#     filename='Files/q8_no_std_errors.png'
# )

## Question 9: Non-stationary problem

Let us now consider a non-stationary problem. Let $\delta = 0.1$ and imagine that after 500 time steps, the parameter of actions 2 and 3 become $0.5+\delta$ and $0.5+2\delta$ respectively. Run $\epsilon$-greedy and UCB for a fixed value of $\alpha = 0.1$ and the averaging value estimation. For $\epsilon$ use values 1/4 and 1/8. Plot only the reward graph as above (you should have 2 lines for each of the $\epsilon$ values, two lines for UCB and one for Thompson sampling). Explain what you see in the graph. Based on
these results, which algorithm is best suited to cope with non-stationarity?

### 9.1. Getting the data

In [32]:
delta = 0.1

nb_runs = 100
nb_steps = 1000

data_Q9 = np.zeros((4, 8, nb_steps, nb_runs))

epsilons = [1 / 4, 1 / 8]
alphas = [None, 0.1]
c = 2

# Run the algorithms
for run_ix in range(nb_runs):
    for epsilon_ix, epsilon in enumerate(epsilons):
        for alpha_ix, alpha in enumerate(alphas):
            q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
            mab_eg = EpsilonGreedy_MAB_Q4(q_star, epsilon=epsilon, alpha=alpha)
            for step_ix in range(nb_steps):
                if step_ix == 500:
                    mab_eg.q_star = [0.5, 0.5 + delta, 0.5 + 2 * delta]
                mab_eg.draw_sample()
            data_Q9[0, epsilon_ix * 2 + alpha_ix, :, run_ix] = mab_eg.mean_reward_history
            data_Q9[1, epsilon_ix * 2 + alpha_ix, :, run_ix] = mab_eg.fraction_correct_history
            data_Q9[2, epsilon_ix * 2 + alpha_ix, :, run_ix] = mab_eg.instant_regret_history
            data_Q9[3, epsilon_ix * 2 + alpha_ix, :, run_ix] = mab_eg.total_regret_history
    for alpha_ix, alpha in enumerate(alphas):
        q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
        mab_ucb = UCB_MAB_Q6(q_star, alpha=alpha, c=c)
        for action_ix in range(len(q_star)):
            mab_ucb.update(action_ix)
        for step_ix in range(nb_steps - len(q_star)):
            if step_ix == 500:
                mab_ucb.q_star = [0.5, 0.5 + delta, 0.5 + 2 * delta]
            mab_ucb.draw_sample()
        data_Q9[0, 4 + alpha_ix, :, run_ix] = mab_ucb.mean_reward_history
        data_Q9[1, 4 + alpha_ix, :, run_ix] = mab_ucb.fraction_correct_history
        data_Q9[2, 4 + alpha_ix, :, run_ix] = mab_ucb.instant_regret_history
        data_Q9[3, 4 + alpha_ix, :, run_ix] = mab_ucb.total_regret_history
    for alpha_ix, alpha in enumerate(alphas):
        q_star = [0.5, 0.5 - delta, 0.5 - 2 * delta]
        mab_thomson = ThomsonSampling_MAB_Q7(q_star, alpha=alpha)
        for step_ix in range(nb_steps):
            if step_ix == 500:
                mab_thomson.q_star = [0.5, 0.5 + delta, 0.5 + 2 * delta]
            mab_thomson.draw_sample()
        data_Q9[0, 6 + alpha_ix, :, run_ix] = mab_thomson.mean_reward_history
        data_Q9[1, 6 + alpha_ix, :, run_ix] = mab_thomson.fraction_correct_history
        data_Q9[2, 6 + alpha_ix, :, run_ix] = mab_thomson.instant_regret_history
        data_Q9[3, 6 + alpha_ix, :, run_ix] = mab_thomson.total_regret_history

### 9.3. Plotting the results

In [ ]:
plot(
    data=data_Q9,
    title="Question 9\nEpsilon Greedy, UCB and Thomson Sampling bandits",
    main_labels=[
        "Time steps",
        "Mean reward",
        "% optimal actions",
        "Instantaneous regret (averaged)",
        "Total regret (averaged)",
    ],
    ax_titles=[
        f"Mean reward and SE averaged over {nb_runs} runs",
        f"Fraction of selected best actions averaged over {nb_runs} runs",
        f"Instantaneous regret averaged over {nb_runs} runs",
        f"Total regret averaged over {nb_runs} runs",
    ],
    algs_info=[
        (rf"Epsilon Greedy, $\epsilon = {epsilons[0]}$, Empirical mean", "tab:blue", 0),
        (rf"Epsilon Greedy, $\epsilon = {epsilons[0]}, \alpha = {alphas[1]}$", "b", 0),
        (rf"Epsilon Greedy, $\epsilon = {epsilons[1]}$, Empirical mean", "tab:green", 0),
        (rf"Epsilon Greedy, $\epsilon = {epsilons[1]}, \alpha = {alphas[1]}$", "g", 0),
        (rf"UCB, Empirical mean, $C = {c}$", "tab:orange", 0),
        (rf"UCB, $\alpha = {alphas[1]}, C = {c}$", "orange", 0),
        (r"Thomson Sampling, Empirical mean", "tab:red", 0),
        (rf"Thomson Sampling, $\alpha = {alphas[1]}$", "r", 0),
    ],
    size=(10, 20),
    range_y=[
        (0, 1),
        (0, 1),
        (0, np.max(np.mean(data_Q9[2, :, :, :], axis=2) * 1.1)),
        (0, np.max(np.mean(data_Q9[3, :, :, :], axis=2) * 1.1)),
    ],
    fill_std=[None, None, None, None],
    filename="q9.png" if SAVE_FILES else None,
    show=SHOW_GRAPHS,
)

In [34]:
# import matplotlib.pyplot as plt
# from matplotlib.transforms import Bbox
# import numpy as np

# # Generate random data
# x = np.random.rand(10)
# y = np.random.rand(10)

# # Create a figure and axes
# fig, ax = plt.subplots(figsize=(8,6))

# # Plot the data
# ax.scatter(x, y, label='Data')

# # Add a legend to the side of the plot
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # Show the plot
# plt.savefig('your_file_name.png', bbox_inches=Bbox([[0, 0], [10, 10]]))